# Análise Astronômica Modular

Este notebook utiliza os módulos Python no diretório `src/` para realizar uma análise de visibilidade astronômica.

In [ ]:
# Importação dos módulos locais
from src.config import *
from src.location import set_location_for_uberaba, set_timezone_for_sao_paulo
from src.targets import load_targets_from_files
from src.analysis import calculate_nightly_events, analyze_target_visibility_for_night
from src.plotting import plot_target_visibility

## 1. Análise Noturna Detalhada

In [ ]:
# Configurar localização e fuso horário
observer_location = set_location_for_uberaba()
observer_timezone = set_timezone_for_sao_paulo()

# Carregar alvos
all_targets = load_targets_from_files()

# Configurar data da análise
analysis_date = date.today()
print(f"\nAnalisando a noite de: {analysis_date.strftime('%Y-%m-%d')}")

# Calcular eventos da noite
night_events = calculate_nightly_events(analysis_date, observer_location, observer_timezone)

# Analisar e plotar cada alvo
if all_targets and night_events.get('evening_astro_twilight'):
    for target_name, target_data in all_targets.items():
        if target_data['coord'] is None and target_data['type'] != 'solar_system':
            print(f"\nAVISO: Coordenadas não encontradas para {target_name}. Pulando.")
            continue
        
        print(f"\n--- Analisando {target_name} ---")
        
        # Para alvos do sistema solar, precisamos calcular a posição para a grade de tempo
        target_coord = target_data['coord']
        if target_data['type'] == 'solar_system':
             night_duration = (night_events['morning_astro_twilight'] - night_events['evening_astro_twilight']).to(u.hour)
             time_grid_night = night_events['evening_astro_twilight'] + np.linspace(0, night_duration.value, 100) * u.hour
             target_coord = get_body(target_data['iau_name'], time_grid_night)

        times_utc, target_altaz = analyze_target_visibility_for_night(
            target_coord,
            night_events['evening_astro_twilight'],
            night_events['morning_astro_twilight'],
            observer_location
        )
        
        if times_utc is not None:
            times_local = [t.to_datetime(observer_timezone) for t in times_utc]
            plot_target_visibility(
                target_name,
                times_local,
                target_altaz,
                night_events,
                observer_location,
                observer_timezone
            )
        else:
            print(f"Não foi possível analisar a visibilidade para {target_name}.")
else:
    print("\nNenhum alvo para analisar ou eventos da noite não puderam ser calculados.")